# Evaluation of Imputation Quality with `imp_eval()` from `missmecha.evaluation`

This notebook demonstrates how to evaluate the quality of imputed data using the `imp_eval()` function from `missmecha.evaluation`.

We show:
- Per-column and overall evaluation metrics
- Support for mixed-type data (numerical and categorical)
- The `AvgERR` concept: a simple but elegant way to automatically dispatch the appropriate evaluation metric based on variable type



### A Note on AvgERR

The idea behind `AvgERR` is to evaluate imputation performance based on variable types:

$
\text{AvgErr}(v_j) =
\begin{cases}
\frac{1}{n} \sum\limits_{i=1}^{n} |X_{ij} - \hat{X}_{ij}|, & \text{if } v_j \text{ is continuous (MAE)} \\\\
\sqrt{\frac{1}{n} \sum\limits_{i=1}^{n} (X_{ij} - \hat{X}_{ij})^2}, & \text{if } v_j \text{ is continuous (RMSE)} \\\\
\frac{1}{n} \sum\limits_{i=1}^{n} (X_{ij} - \hat{X}_{ij})^2, & \text{if } v_j \text{ is continuous (MSE)} \\\\
\frac{1}{n} \sum\limits_{i=1}^{n} \text{Acc}(X_{ij}, \hat{X}_{ij}), & \text{if } v_j \text{ is categorical}
\end{cases}
$


In this implementation, if a `status` dictionary is provided, the function automatically applies the appropriate metric:
- **Numerical columns** use the selected method (RMSE or MAE)
- **Categorical/discrete columns** use classification accuracy

## Setup
Import required packages and the evaluation function.


In [1]:
import pandas as pd
import numpy as np
from missmecha.evaluation import imp_eval

### Create fully observed mixed-type dataset


In [2]:
df_true = pd.DataFrame({
    "age": [25, 30, 22, 40, 35, 50],
    "income": [3000, 4500, 2800, 5200, 4100, 6000],
    "gender": ["M", "F", "M", "F", "F", "M"],
    "job_level": ["junior", "mid", "junior", "senior", "mid", "senior"]
})
df_true

,age,income,gender,job_level
0,25,3000,M,junior
1,30,4500,F,mid
2,22,2800,M,junior
3,40,5200,F,senior
4,35,4100,F,mid
5,50,6000,M,senior


### Inject missing values

In [3]:
df_incomplete = df_true.copy()
df_incomplete.loc[1, "age"] = np.nan
df_incomplete.loc[2, "income"] = np.nan
df_incomplete.loc[3, "gender"] = np.nan
df_incomplete.loc[4, "job_level"] = np.nan
df_incomplete

,age,income,gender,job_level
0,25.0,3000.0,M,junior
1,NaN,4500.0,F,mid
2,22.0,NaN,M,junior
3,40.0,5200.0,NaN,senior
4,35.0,4100.0,F,NaN
5,50.0,6000.0,M,senior


### Impute missing values (integer mean for numeric, mode for categorical)

In [4]:
df_filled = df_incomplete.copy()

for col in df_filled.columns:
    if df_filled[col].dtype.kind in "iufc":
        df_filled[col] = df_filled[col].fillna(round(df_filled[col].mean()))
    else:
        df_filled[col] = df_filled[col].fillna(df_filled[col].mode()[0])

df_filled

,age,income,gender,job_level
0,25.0,3000.0,M,junior
1,34.0,4500.0,F,mid
2,22.0,4560.0,M,junior
3,40.0,5200.0,M,senior
4,35.0,4100.0,F,junior
5,50.0,6000.0,M,senior


### Define variable types


In [5]:
status = {
    "age": "num",
    "income": "num",
    "gender": "cat",
    "job_level": "disc"
}


### Run `imp_eval()` with AvgERR logi



In [6]:
results = imp_eval(
    ground_truth=df_true,
    filled_df=df_filled,
    incomplete_df=df_incomplete,
    method="mae",  # used for numerical columns
    status=status
)

In [9]:
print("Column-wise scores:")
for k, v in results["column_scores"].items():
    print(f"  {k}: {v:.2f}")

print(f"\n Overall score: {results['overall_score']:.2f}")

Column-wise scores:
  age: 4.00
  income: 1760.00
  gender: 0.00
  job_level: 0.00

 Overall score: 441.00
